In [ ]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import os

# Scrapping Data from Github OWID Datasets

In [ ]:
import bs4
import requests

In [ ]:
url = "https://github.com/owid/owid-datasets/tree/master/datasets"

webpage = requests.get(url).text

In [ ]:
soup = bs4.BeautifulSoup(webpage)

In [ ]:
results = soup.find_all('a', {'class':'js-navigation-open Link--primary'})

links =[]
titles = []
for result in results:
  rel_short_link = result['href'].split('/')[-1]
  data_url = 'https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/' + rel_short_link+ '/'+rel_short_link+'.csv'
  
  titles.append("_".join(result['title'].split(" ")))
  links.append(data_url) 
# class="js-navigation-open Link--primary"

In [ ]:
df = pd.read_csv(links[-1])

In [ ]:
titles[:5]

['_Literacy_by_years_of_schooling_US_1947_–_OECD_(2014)_',
 '_Literate_world_population_(OurWorldInData_based_on_OECD_and_UNESCO)',
 '20th_century_deaths_in_US_-_CDC',
 'A_Century_of_Work_and_Leisure_-_Ramey_and_Francis_(2009)',
 'Abortion_rates_(per_1000_women)_-_Sedgh_et_al._(2016)']

# Using SQL and Unit testing



In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
conn.close()

In [ ]:
conn = sqlite3.connect("OWID.db")
for title, link in zip(titles, links):
  df = pd.read_csv(link)
  df.to_sql(title, conn, index=False)
  print(title)
conn.close()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
conn = sqlite3.connect("OWID.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [ ]:
len(tables)

993

In [ ]:
pd.read_sql( 'SELECT * FROM "%s" '  % (tables[-1][0]), conn)

,Entity,Year,Youth mortality rates (0-14)
0,Afghanistan,1977,297.48592
1,Afghanistan,1978,289.08271
2,Afghanistan,1979,280.57551
3,Afghanistan,1980,272.02735
4,Afghanistan,1981,263.45558
...,...,...,...
8300,Zimbabwe,2013,80.98464
8301,Zimbabwe,2014,75.18350
8302,Zimbabwe,2015,70.86210
8303,Zimbabwe,2016,66.93248


In [ ]:
pd.read_sql( "SELECT * FROM %s"  % ('"' +tables[2][0] + '"'), conn)

,Entity,Year,Accidents (excl. road) - Deaths,Accidents (total) - Deaths,Arteriosclerosis - Deaths,Bronchitis - Deaths,Cancers - Deaths,COPD - Deaths,Dementia - Deaths,Diabetes - Deaths,Diarrheal disease - Deaths,Heart disease - Deaths,Homicide - Deaths,Kidney infection - Deaths,Liver disease - Deaths,Neonatal disorders - Deaths,Nephritis - Deaths,Other causes - Deaths,Pneumonia and influenza - Deaths,Respiratory disease - Deaths,Road accidents - Deaths,Septicemia - Deaths,Stroke - Deaths,Suicide - Deaths,Syphilis - Deaths,Total deaths - Deaths,Tuberculosis - Deaths,Non-communicable diseases (NCDs) - Deaths,"Communicable, infectious, neonatal and other deaths - Deaths",Accidents (excl. road) - Death Rates,Accidents (total) - Death Rates,Arteriosclerosis - Death Rates,Bronchitis - Death Rates,Cancers - Death Rates,COPD - Death Rates,Dementia - Death Rates,Diabetes - Death Rates,Diarrheal disease - Death Rates,Heart disease - Death Rates,Homicide - Death Rates,...,Other causes - Death Rates,Pneumonia and influenza - Death Rates,Respiratory disease - Death Rates,Road accidents - Death Rates,Septicemia - Death Rates,Stroke - Death Rates,Suicide - Death Rates,Syphilis - Death Rates,Total deaths - Death Rates,Tuberculosis - Death Rates,Non-communicable diseases (NCDs) - Death Rates,"Communicable, infectious, neonatal and other deaths - Death Rates",Accidents (excl. road) - % of Deaths,Accidents (total) - % of Deaths,Arteriosclerosis - % of Deaths,Bronchitis - % of Deaths,Cancers - % of Deaths,COPD - % of Deaths,Dementia - % of Deaths,Diabetes - % of Deaths,Diarrheal disease - % of Deaths,Heart disease - % of Deaths,Homicide - % of Deaths,Kidney infection - % of Deaths,Liver disease - % of Deaths,Neonatal disorders - % of Deaths,Nephritis - % of Deaths,Other bronchopulmonic diseases - % of Deaths,Other causes - % of Deaths,Pneumonia and influenza - % of Deaths,Respiratory disease - % of Deaths,Road accidents - % of Deaths,Septicemia - % of Deaths,Stroke - % of Deaths,Suicide - % of Deaths,Syphilis - % of Deaths,Total deaths - % of Deaths,Tuberculosis - % of Deaths,Non-communicable diseases (NCDs) - % of Deaths,"Communicable, infectious, neonatal and other deaths - % of Deaths"
0,United States,1900,14429.0,14429.0,NaN,NaN,12769,NaN,10015.0,NaN,28491.0,27427,NaN,NaN,NaN,NaN,17699.0,NaN,40362,NaN,NaN,NaN,21353.0,NaN,NaN,343217,38820.0,61549,267239,72.3,72.3,NaN,NaN,64.0,NaN,50.2,NaN,142.7,137.4,NaN,...,NaN,202.2,NaN,NaN,NaN,106.9,NaN,NaN,1719.1,194.4,308.3,1338.5,4.20,4.20,NaN,NaN,3.72,NaN,2.92,NaN,8.30,7.99,NaN,NaN,NaN,NaN,5.16,NaN,NaN,11.76,NaN,NaN,NaN,6.22,NaN,NaN,100.0,11.31,17.93,77.86
1,United States,1901,16958.0,NaN,NaN,8171.0,13438,NaN,9771.0,NaN,23986.0,28329,NaN,NaN,NaN,NaN,18195.0,NaN,39908,NaN,NaN,NaN,21639.0,NaN,NaN,332203,38434.0,63406,251839,83.8,83.8,NaN,40.4,66.4,NaN,48.3,NaN,118.5,140.0,NaN,...,NaN,197.2,NaN,NaN,NaN,106.9,NaN,NaN,1641.5,189.9,313.3,1244.4,5.10,NaN,NaN,2.46,4.05,NaN,2.94,NaN,7.22,8.53,NaN,NaN,NaN,NaN,5.48,NaN,NaN,12.01,NaN,NaN,NaN,6.51,NaN,NaN,100.0,11.57,19.09,75.81
2,United States,1902,14915.0,NaN,NaN,8112.0,13653,NaN,9309.0,NaN,21599.0,29925,NaN,NaN,NaN,NaN,18642.0,NaN,33208,NaN,NaN,NaN,21391.0,NaN,NaN,318636,35859.0,64969,238752,72.5,72.5,NaN,39.4,66.3,NaN,45.2,NaN,104.9,145.4,NaN,...,NaN,161.3,NaN,NaN,NaN,103.9,NaN,NaN,1548.1,174.2,315.6,1160.0,4.68,NaN,NaN,2.55,4.28,NaN,2.92,NaN,6.78,9.39,NaN,NaN,NaN,NaN,5.85,NaN,NaN,10.42,NaN,NaN,NaN,6.71,NaN,NaN,100.0,11.25,20.39,74.93
3,United States,1903,17050.0,NaN,NaN,7508.0,14650,NaN,8604.0,NaN,20997.0,31797,NaN,NaN,NaN,NaN,20158.0,NaN,35455,NaN,NaN,NaN,22031.0,NaN,NaN,327295,37102.0,68478,241767,81.4,81.4,NaN,35.8,70.0,NaN,41.1,NaN,100.3,151.8,NaN,...,NaN,169.3,NaN,NaN,NaN,105.2,NaN,NaN,1562.8,177.2,327.0,1154.4,5.21,NaN,NaN,2.29,4.48,NaN,2.63,NaN,6.42,9.72,NaN,NaN,NaN,NaN,6.16,NaN,NaN,10.83,NaN,NaN,NaN,6.73,NaN,NaN,100.0,11.34,20.92,73.87
4,United States,1904,18228.0,NaN,NaN,7571.0,15247,NaN,8702.0,NaN,23791.0,34925,NaN,NaN,NaN,7569.0,21851.0,NaN,40989,NaN

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def main():
    database = r"test.db"

    sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        begin_date text,
                                        end_date text
                                    ); """

    sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    priority integer,
                                    status_id integer NOT NULL,
                                    project_id integer NOT NULL,
                                    begin_date text NOT NULL,
                                    end_date text NOT NULL,
                                    FOREIGN KEY (project_id) REFERENCES projects (id)
                                );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_projects_table)

        # create tasks table
        create_table(conn, sql_create_tasks_table)
    else:
        print("Error! cannot create the database connection.")